# dataset测试

In [19]:
from dataset import ModelNetDataLoader
from torch.utils.data import DataLoader

DATA_PATH = 'data/modelnet40_normal_resampled/'
TRAIN_DATASET = ModelNetDataLoader(
    root=DATA_PATH, npoint=1024, 
    split='test' , normal_channel=True, classes=40)

#os.environ["CUDA_VISIBLE_DEVICES"] = '0'
print(len(TRAIN_DATASET))
print(len(TRAIN_DATASET[500][0]))
print(len(TRAIN_DATASET[500][1]))
TRAIN_DATASET[500][1]

The size of test data is 2468
2468
1024
1


array([7], dtype=int32)

In [2]:
from visualizer.show3d_balls import showpoints,cmap
# from ShapeNetDataLoader import PartNormalDataset
root = '../data/shapenetcore_partanno_segmentation_benchmark_v0_normal/'
dataset = ModelNetDataLoader(root = root, npoints=2048, split='test', normal_channel=False)
idx = np.random.randint(0, len(dataset))
data = dataset[idx]
point_set, _, seg = data
choice = np.random.choice(point_set.shape[0], 2048, replace=True)
point_set, seg = point_set[choice, :], seg[choice]
seg = seg - seg.min()
gt = cmap[seg, :]
pred = cmap[seg, :]
showpoints(point_set, gt, c_pred=pred, waittime=0, showrot=False, magnifyBlue=0, freezerot=False,
            background=(255, 255, 255), normalizecolor=True, ballradius=10)

ModuleNotFoundError: No module named 'cv2'

In [20]:
TRAIN_DATASET[500]

(array([[ 0.5606077 , -0.20170672,  0.27205136,  0.6131    ,  0.5877    ,
         -0.5279    ],
        [-0.97924536, -0.00238999, -0.19810769, -0.9647    ,  0.05318   ,
         -0.2579    ],
        [-0.05963943,  0.3176909 , -0.25163224,  0.02736   ,  0.9462    ,
         -0.3223    ],
        ...,
        [ 0.40093103,  0.16140316,  0.20816076,  0.4113    ,  0.907     ,
          0.0902    ],
        [-0.94465876,  0.0843557 , -0.11069418, -0.9627    ,  0.06346   ,
         -0.2628    ],
        [-0.7020539 , -0.16849558, -0.33485946, -0.9122    , -0.4097    ,
          0.003455  ]], dtype=float32),
 array([7], dtype=int32))

In [21]:
trainDataLoader = DataLoader(
        TRAIN_DATASET , batch_size=4, 
        shuffle=True  , num_workers=8)
 #遍历
len(trainDataLoader)

617

In [25]:
train_features, train_labels = next(iter(trainDataLoader))
print(train_features.shape) 
print(train_labels.shape) 
train_labels

torch.Size([4, 1024, 6])
torch.Size([4, 1])


tensor([[21],
        [28],
        [38],
        [17]], dtype=torch.int32)

In [7]:
from tqdm import tqdm


In [12]:
trainDataLoader = tqdm(trainDataLoader)

  0%|          | 0/227 [00:00<?, ?it/s]

In [17]:
from train_cls import *
# cfg.num_point = 1024
# model = getattr(importlib.import_module('models.{}.model'.format('Hengshuang')), 'PointTransformerCls')(cfg).cuda()

In [24]:
target = train_labels[:,0]
target

tensor([2, 4, 2, 8], dtype=torch.int32)

In [23]:
np.unique(target)

array([2, 4, 8], dtype=int32)

In [25]:
target==2

tensor([ True, False,  True, False])

In [26]:
cat_idex = (target==2)
cat_idex

tensor([ True, False,  True, False])

In [27]:
target[cat_idex].long().data

tensor([2, 2])

In [30]:
cat_idex.sum().item()

2

In [35]:
pred = torch.Tensor([1,2])


In [36]:
pred.eq(target[cat_idex].long().data)

tensor([False,  True])

In [41]:
train_features[target==2].size()

torch.Size([2, 1024, 6])

In [40]:
train_features[target==2].size()[0]

2

In [ ]:
import random
import math
import matplotlib.pyplot as plt

M = 50 # 所有的点数量
N = 10 # fps采样点数量
x = []
y = []
xy = []

# 生成数据
random.seed(0)
for _ in range(M):
    px = round(random.uniform(1, 10), 2)  # 生成随机数，浮点类型 控制2位精度
    py = round(random.uniform(1, 10), 2)
    x.append(px)
    y.append(py)
    xy.append((px, py))

xy

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.title('init_points')
plt.scatter(x, y, marker='o', c='r', edgecolors='b', s=200)

In [ ]:
select = []
rest = [num for num in range(0, M)]
max_dist = -99999
farthest_point = 99999

In [ ]:
# step 1 随机选择一个点
random.seed(1)
ind = random.randint(0, M - 1)

select.append(ind)
rest.remove(ind)
# print('select', select)
# print('rest', rest)

In [17]:
import numpy as np
import torch
a=torch.Tensor([1,2,3,4,5,6,6])
b=torch.Tensor([1,2,2,2,5,6,8])
print(np.unique(a))
print((a==6).sum()/2)
b[a==6]

[1. 2. 3. 4. 5. 6.]
tensor(1.)


tensor([6., 8.])

In [1]:
import torch

fc = torch.nn.Linear(3, 5)
params = list(fc.named_parameters())
print(params.__len__())
print(params[0])
print(params[1])

2
('weight', Parameter containing:
tensor([[-0.5754,  0.5086, -0.4083],
        [ 0.4169,  0.0558,  0.4164],
        [-0.3582, -0.0306,  0.1457],
        [ 0.1982, -0.2772,  0.3499],
        [ 0.2818,  0.4288,  0.0650]], requires_grad=True))
('bias', Parameter containing:
tensor([ 0.1805, -0.2897, -0.5245,  0.5769, -0.3859], requires_grad=True))


In [3]:
import torch.nn as nn
mlp = torch.nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(1024, 256),
            nn.Linear(256, 64),
            nn.Linear(64, 16),
            nn.Linear(16, 1)
        )
params = list(mlp.named_parameters())
print(params.__len__())

print(params[0])
print(params[1])

print(params[2])
print(params[3])

8
('1.weight', Parameter containing:
tensor([[ 0.0292,  0.0305,  0.0092,  ...,  0.0144,  0.0078, -0.0016],
        [-0.0066,  0.0309,  0.0013,  ...,  0.0225, -0.0079,  0.0265],
        [ 0.0103, -0.0150, -0.0131,  ..., -0.0169,  0.0047, -0.0160],
        ...,
        [-0.0130, -0.0046, -0.0248,  ..., -0.0183, -0.0054, -0.0208],
        [ 0.0249, -0.0305,  0.0288,  ...,  0.0077,  0.0029, -0.0243],
        [ 0.0132,  0.0310,  0.0038,  ..., -0.0278, -0.0055,  0.0221]],
       requires_grad=True))
('1.bias', Parameter containing:
tensor([ 9.8564e-04,  1.1425e-02,  2.7055e-03, -2.9118e-02,  3.2956e-03,
        -9.0277e-03, -3.0123e-03, -3.3727e-03, -2.2414e-02,  1.7847e-02,
         1.6348e-02, -1.2228e-02,  3.5869e-03, -2.1708e-02,  2.6500e-03,
         2.4879e-02,  2.2134e-02,  2.7580e-02,  2.2203e-02, -2.8633e-02,
         2.0920e-03, -1.6087e-02,  1.6634e-02,  2.1290e-02, -1.1951e-03,
        -3.0302e-02,  2.0438e-02,  2.8376e-02, -2.3913e-02,  9.7344e-04,
         2.6216e-02,  2.2986e-

In [4]:
import torchvision.models as models

In [5]:
model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(models.__dict__[name]))
model_names

['alexnet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'googlenet',
 'inception_v3',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet_v2',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'wide_resnet101_2',
 'wide_resnet50_2']

In [6]:
torch.backends.cudnn.version()

8005

In [7]:
torch.backends.cudnn.enabled

True

In [ ]:
# print(os.getcwd())
# print(__file__)
# raise NotImplementedError()